In [1]:
import pandas as pd

In [2]:
total_df = pd.read_csv('../data1/total_df.csv')
review_df = pd.read_csv('../data2/review_df.csv')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

In [5]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   merchant     225 non-null    object 
 1   category     225 non-null    object 
 2   star         225 non-null    float64
 3   address      225 non-null    object 
 4   oper_time    225 non-null    object 
 5   starCount    225 non-null    int64  
 6   reviewCount  225 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 12.4+ KB


In [4]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22383 entries, 0 to 22382
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   merchant      22383 non-null  object 
 1   user_name     22361 non-null  object 
 2   user_rank     22383 non-null  int64  
 3   num_response  22383 non-null  object 
 4   user_star     22383 non-null  float64
 5   time          22383 non-null  object 
 6   rating        22383 non-null  float64
 7   content       18674 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 1.4+ MB


In [4]:
# 결측치 제거 및 전처리
review_df = review_df.dropna(subset=['content'])
review_df['cleaned_content'] = review_df['content'].str.lower().replace('[^a-zA-Z가-힣]', ' ', regex=True)

In [9]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000, min_df=5)
X_tfidf = vectorizer.fit_transform(review_df['cleaned_content'])

In [10]:
# 군집화
kmeans = KMeans(n_clusters=8, random_state=42)
clusters = kmeans.fit_predict(X_tfidf)

In [11]:
# 클러스터 결과를 리뷰 데이터프레임에 추가
review_df['cluster'] = clusters

In [16]:
# 음식점별로 가장 많이 속한 클러스터의 가중치를 계산
cluster_weights = review_df.groupby(['merchant', 'cluster']).size().unstack(fill_value=0)
cluster_weights = normalize(cluster_weights, norm='l1')

In [17]:
# 가중치 데이터프레임 생성
cluster_weights_df = pd.DataFrame(cluster_weights, index=cluster_weights.index, columns=kmeans.cluster_centers_.argsort()[:, ::-1][0])

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [18]:
# 가중치 데이터프레임 생성
cluster_indices = review_df.groupby(['merchant', 'cluster']).size().unstack(fill_value=0).index
cluster_columns = ['cluster_' + str(i) for i in range(kmeans.n_clusters)]
cluster_weights_df = pd.DataFrame(cluster_weights, index=cluster_indices, columns=cluster_columns)

# total_df에 가중치 정보를 병합
total_df = total_df.merge(cluster_weights_df, how='left', on='merchant')

In [27]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   merchant     225 non-null    object 
 1   category     225 non-null    object 
 2   star         225 non-null    float64
 3   address      225 non-null    object 
 4   oper_time    225 non-null    object 
 5   starCount    225 non-null    int64  
 6   reviewCount  225 non-null    int64  
 7   cluster_0    198 non-null    float64
 8   cluster_1    198 non-null    float64
 9   cluster_2    198 non-null    float64
 10  cluster_3    198 non-null    float64
 11  cluster_4    198 non-null    float64
 12  cluster_5    198 non-null    float64
 13  cluster_6    198 non-null    float64
 14  cluster_7    198 non-null    float64
dtypes: float64(9), int64(2), object(4)
memory usage: 26.5+ KB


In [31]:
total_df['cluster_1'].unique()

array([0.80751174, 0.76708075, 0.80645161, 0.79577465, 0.73469388,
       0.80357143, 0.6875    , 0.61555076, 0.67283951,        nan,
       0.83333333, 0.69      , 0.89449541, 0.68342857, 0.81549815,
       0.74311927, 0.75460123, 0.73873874, 0.7593985 , 0.65753425,
       0.82278481, 0.78776978, 0.80962343, 0.63157895, 0.82758621,
       0.85106383, 0.82142857, 0.52893082, 0.80952381, 0.90697674,
       0.79183673, 0.75510204, 0.79054054, 0.79569892, 0.70955882,
       0.86764706, 0.74611399, 0.81818182, 0.85294118, 0.85714286,
       0.86585366, 0.86206897, 0.84042553, 1.        , 0.68181818,
       0.8030303 , 0.67924528, 0.78787879, 0.72727273, 0.8       ,
       0.8028169 , 0.7578125 , 0.89230769, 0.85416667, 0.78688525,
       0.54716981, 0.73333333, 0.86956522, 0.625     , 0.87179487,
       0.9       , 0.8677686 , 0.66666667, 0.94270833, 0.63114754,
       0.74380165, 0.75      , 0.82051282, 0.7699115 , 0.7027027 ,
       0.64951768, 0.75609756, 0.88636364, 0.76666667, 0.69565

In [29]:
cluster_indices

Index(['1984', '21세기 우리바다수산', '38도씨식당', '943킹스크로스', '가로수포차', '가미우동', '가비애',
       '감성타코 홍대점', '강강술래 홍대점', '개화기요정',
       ...
       '홍대씨부엉', '홍대인파스타', '홍대조폭떡볶이 홍대2호점', '홍대차칸곱창', '홍스쭈꾸미 홍대본점',
       '홍익닭한마리 홍대본점', '황곱 홍대점', '효봉포차', '후라토식당 상수직영점', '훠궈나라 홍대점'],
      dtype='object', name='merchant', length=198)

In [32]:
# 각 클러스터 중심점에 대해 가장 높은 TF-IDF 값을 가진 단어 추출
feature_names = vectorizer.get_feature_names_out()
top_n = 10  # 각 클러스터별로 추출할 상위 단어 수

cluster_terms = {}
for i, centroid in enumerate(kmeans.cluster_centers_):
    sorted_indices = centroid.argsort()[::-1][:top_n]
    cluster_terms[f'Cluster {i}'] = [feature_names[idx] for idx in sorted_indices]

# DataFrame 생성
cluster_terms_df = pd.DataFrame(cluster_terms)
cluster_terms_df


,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7
0,너무,좋아요,맛있어요,곱창,진짜,같습니다,맛은,파스타
1,맛있어요,정말,너무,맛있어요,맛있어요,너무,그냥,피자
2,맛있고,맛있어요,친절하고,너무,너무,좋은,너무,스테이크
3,좋아요,맛있고,완전,홍대,여기,정말,있는데,모두
4,좋고,그냥,좋고,대창,맛있음,조금,근데,맛집
5,맛있게,맛있음,정말,진짜,맛있고,좋을,조금,맛있었어요
6,분위기도,사장님,너무너무,나오는,최고,그냥,비해,크림
7,맛있어서,맛이,츠케멘,해물탕도,맛있었어요,가게,양이,맛있고
8,친절하시고,분위기,다음에,홍대에서,엄청,맛있었어요,정말,맛있어요
9,정말,좋고,좋아요,막창,그냥,맛은,진짜,너무


In [33]:
review_df

,merchant,user_name,user_rank,num_response,user_star,time,rating,content,cleaned_content,cluster
0,1984,2018.05.10,22,58,2.5,2024.04.21.,5.0,"까페에서 책 읽기를 좋아하는 사람인데요. 매우 쾌적하고 음악으로, 다른 손님 때문에...",까페에서 책 읽기를 좋아하는 사람인데요 매우 쾌적하고 음악으로 다른 손님 때문에...,1
2,1984,맛난곳만리뷰,7,7,5.0,2024.02.10.,5.0,다양한 볼거리들 있어서 좋아요 분위기도 괜찮구여,다양한 볼거리들 있어서 좋아요 분위기도 괜찮구여,1
3,1984,진아,25,58,4.6,2024.02.05.,4.0,넓고 음료 ㄱㅊ앉아있기 좋음뭐 살 것도 있음,넓고 음료 앉아있기 좋음뭐 살 것도 있음,1
4,1984,박찬찬,27,71,3.9,2023.12.16.,1.0,"분위기는 좋으나, 장사할 생각 없어보임커피값도 비싼 편인데 돈값 못함카페는 부수적인...",분위기는 좋으나 장사할 생각 없어보임커피값도 비싼 편인데 돈값 못함카페는 부수적인...,1
5,1984,프로페셔널,16,37,2.7,2023.12.04.,1.0,작년까지 아메리카노도 수준급으로 맛있었는데 왜이리 변했나요 .. 손님이 오면 인사는...,작년까지 아메리카노도 수준급으로 맛있었는데 왜이리 변했나요 손님이 오면 인사는...,1
...,...,...,...,...,...,...,...,...,...,...
22378,훠궈나라 홍대점,ㅇㅅㅇ,31,122,4.0,2019.12.28.,5.0,역대급 최고인데여?????,역대급 최고인데여,1
22379,훠궈나라 홍대점,상준,3,2,5.0,2019.12.14.,5.0,굿굿굿,굿굿굿,1
22380,훠궈나라 홍대점,.,9,15,4.3,2019.11.18.,5.0,맛있어요. 몇번왔어요. 생각나는 맛이에요ㅜㅠ,맛있어요 몇번왔어요 생각나는 맛이에요,1
22381,훠궈나라 홍대점,김후연,2,1,1.0,2019.11.03.,1.0,맛없음,맛없음,1


In [ ]:
# ------------------------------------------------------

In [17]:
import pandas as pd

In [18]:
total_df = pd.read_csv('../data1/total_df.csv')
review_df = pd.read_csv('../data2/review_df.csv')

In [3]:
# !pip install konlpy

In [19]:
from konlpy.tag import Okt
import re

## 단어 추출
# Initialize the Okt object
okt = Okt()

# Remove rows where 'content' is NaN
review_df_clean = review_df.dropna(subset=['content'])

# Function to preprocess and extract keywords from text
def preprocess_text(text):
    # Normalize and remove non-alphabetic characters
    text = re.sub("[^가-힣a-zA-Z ]", "", text)
    # Tokenize and filter out stopwords and extract nouns
    tokens = okt.nouns(text)  # Focusing on nouns for keyword extraction
    tokens = [n for n in tokens if len(n) > 1]
    return tokens

# Apply text processing to the reviews
review_df_clean['keywords'] = review_df_clean['content'].apply(preprocess_text)

# Display the processed keywords
review_df_clean[['content', 'keywords']].head()

C:\Users\Jws\AppData\Local\Temp\ipykernel_32960\2900134844.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df_clean['keywords'] = review_df_clean['content'].apply(preprocess_text)


,content,keywords
0,"까페에서 책 읽기를 좋아하는 사람인데요. 매우 쾌적하고 음악으로, 다른 손님 때문에...","[까페, 사람, 매우, 음악, 다른, 손님, 때문, 방해, 한번, 커피, 메뉴, 리..."
2,다양한 볼거리들 있어서 좋아요 분위기도 괜찮구여,"[볼거리, 분위기]"
3,넓고 음료 ㄱㅊ앉아있기 좋음뭐 살 것도 있음,[음료]
4,"분위기는 좋으나, 장사할 생각 없어보임커피값도 비싼 편인데 돈값 못함카페는 부수적인...","[분위기, 장사, 생각, 보임, 커피, 돈값, 카페, 부수, 수입]"
5,작년까지 아메리카노도 수준급으로 맛있었는데 왜이리 변했나요 .. 손님이 오면 인사는...,"[작년, 아메리카노, 수준, 손님, 인사, 커녕, 인사, 해도, 대충, 사장]"


In [20]:
review_df_clean

,merchant,user_name,user_rank,num_response,user_star,time,rating,content,keywords
0,1984,2018.05.10,22,58,2.5,2024.04.21.,5.0,"까페에서 책 읽기를 좋아하는 사람인데요. 매우 쾌적하고 음악으로, 다른 손님 때문에...","[까페, 사람, 매우, 음악, 다른, 손님, 때문, 방해, 한번, 커피, 메뉴, 리..."
2,1984,맛난곳만리뷰,7,7,5.0,2024.02.10.,5.0,다양한 볼거리들 있어서 좋아요 분위기도 괜찮구여,"[볼거리, 분위기]"
3,1984,진아,25,58,4.6,2024.02.05.,4.0,넓고 음료 ㄱㅊ앉아있기 좋음뭐 살 것도 있음,[음료]
4,1984,박찬찬,27,71,3.9,2023.12.16.,1.0,"분위기는 좋으나, 장사할 생각 없어보임커피값도 비싼 편인데 돈값 못함카페는 부수적인...","[분위기, 장사, 생각, 보임, 커피, 돈값, 카페, 부수, 수입]"
5,1984,프로페셔널,16,37,2.7,2023.12.04.,1.0,작년까지 아메리카노도 수준급으로 맛있었는데 왜이리 변했나요 .. 손님이 오면 인사는...,"[작년, 아메리카노, 수준, 손님, 인사, 커녕, 인사, 해도, 대충, 사장]"
...,...,...,...,...,...,...,...,...,...
22378,훠궈나라 홍대점,ㅇㅅㅇ,31,122,4.0,2019.12.28.,5.0,역대급 최고인데여?????,"[역대, 최고]"
22379,훠궈나라 홍대점,상준,3,2,5.0,2019.12.14.,5.0,굿굿굿,[굿굿굿]
22380,훠궈나라 홍대점,.,9,15,4.3,2019.11.18.,5.0,맛있어요. 몇번왔어요. 생각나는 맛이에요ㅜㅠ,[몇번]
22381,훠궈나라 홍대점,김후연,2,1,1.0,2019.11.03.,1.0,맛없음,[]


In [22]:
from collections import Counter
# Flatten the list of words and count frequencies
all_words = [word for sublist in review_df_clean['keywords'] for word in sublist]


word_counts = Counter(all_words)

# Display the most common words
word_counts.most_common(20)

[('진짜', 1822),
 ('직원', 1740),
 ('분위기', 1550),
 ('방문', 1327),
 ('가격', 1287),
 ('정말', 1082),
 ('음식', 1002),
 ('사장', 972),
 ('홍대', 958),
 ('피자', 956),
 ('맛집', 933),
 ('사람', 908),
 ('라멘', 887),
 ('그냥', 877),
 ('정도', 875),
 ('여기', 874),
 ('서비스', 819),
 ('최고', 816),
 ('추천', 807),
 ('생각', 802)]

In [23]:
a = '아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'

In [24]:
a = a.replace(" ", ",")

In [25]:
## 불용어 제거
# Define a list of common Korean stopwords
stopwords = [a,'맛있어요', '맛있고', '좋아요', '맛', '맛은', '맛이', '좋고', '안', '엄청', '맛있게', '맛도', '다른', '먹고', 
    '분위기', '맛있음', '맛있었어요', '같아요', '여기', '그리고','여기', '맛', '너무', '진짜', '다', '좀', '정말', '잘', '더', '그냥', '또', '것', '수', '많이', '꼭', '이', '거', '것은', '에서', '에는', 
    '있습니다', '있어', '있는', '입니다', '하는', '하기', '하고', '했어요', '했습니다', '한', '할', '합니다', '해', '해요', 
    '으로', '로는', '와', '과', '도', '고', '에', '에게', '와서', '와의', '으로부터', '이다', '있다', '되다', '같다']

# Filter out stopwords from the word lists
review_df_clean['filtered_words'] = review_df_clean['keywords'].apply(lambda words: [word for word in words if word not in stopwords])

# Recalculate word frequencies without stopwords
filtered_words = [word for sublist in review_df_clean['filtered_words'] for word in sublist]
filtered_word_counts = Counter(filtered_words)

# Display the most common words after removing stopwords
filtered_word_counts.most_common(20)

C:\Users\Jws\AppData\Local\Temp\ipykernel_32960\3635285602.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df_clean['filtered_words'] = review_df_clean['keywords'].apply(lambda words: [word for word in words if word not in stopwords])


[('직원', 1740),
 ('방문', 1327),
 ('가격', 1287),
 ('음식', 1002),
 ('사장', 972),
 ('홍대', 958),
 ('피자', 956),
 ('맛집', 933),
 ('사람', 908),
 ('라멘', 887),
 ('정도', 875),
 ('서비스', 819),
 ('최고', 816),
 ('추천', 807),
 ('생각', 802),
 ('곱창', 708),
 ('메뉴', 705),
 ('시간', 692),
 ('웨이', 680),
 ('느낌', 669)]

In [26]:
filtered_word_counts

Counter({'직원': 1740,
         '방문': 1327,
         '가격': 1287,
         '음식': 1002,
         '사장': 972,
         '홍대': 958,
         '피자': 956,
         '맛집': 933,
         '사람': 908,
         '라멘': 887,
         '정도': 875,
         '서비스': 819,
         '최고': 816,
         '추천': 807,
         '생각': 802,
         '곱창': 708,
         '메뉴': 705,
         '시간': 692,
         '웨이': 680,
         '느낌': 669,
         '별로': 669,
         '가게': 658,
         '국물': 653,
         '처음': 642,
         '고기': 623,
         '우동': 607,
         '카페': 562,
         '주문': 549,
         '커피': 544,
         '하나': 510,
         '손님': 507,
         '가성': 501,
         '존맛': 501,
         '조금': 496,
         '테이블': 484,
         '기분': 483,
         '다음': 468,
         '튀김': 442,
         '치즈': 431,
         '친구': 427,
         '추가': 423,
         '파스타': 415,
         '리뷰': 406,
         '다시': 404,
         '자주': 386,
         '딸기': 386,
         '케이크': 383,
         '음료': 381,
         '양도': 378,
         '소스

In [27]:
filtered_words

['까페',
 '사람',
 '매우',
 '음악',
 '손님',
 '때문',
 '방해',
 '한번',
 '커피',
 '메뉴',
 '리필',
 '때문',
 '사실',
 '시간',
 '합리',
 '가격',
 '생각',
 '직원',
 '자주',
 '생각',
 '오히려',
 '손님',
 '방해',
 '공간',
 '문화',
 '노력',
 '세심',
 '웃음',
 '응대',
 '프랜차이즈',
 '공간',
 '합리',
 '가격',
 '홍대',
 '사색',
 '공간',
 '운영',
 '전하',
 '볼거리',
 '음료',
 '장사',
 '생각',
 '보임',
 '커피',
 '돈값',
 '카페',
 '부수',
 '수입',
 '작년',
 '아메리카노',
 '수준',
 '손님',
 '인사',
 '커녕',
 '인사',
 '해도',
 '대충',
 '사장',
 '디저트',
 '커피',
 '무난',
 '복합',
 '공간',
 '쇼핑',
 '힙하',
 '여유',
 '기도',
 '커피',
 '아메리카노',
 '무료',
 '리필',
 '홍대',
 '작업',
 '카페',
 '종류',
 '구경',
 '커피',
 '북카페',
 '편집',
 '느낌',
 '비치',
 '구매',
 '커피',
 '커피',
 '샌드위치',
 '가격',
 '직원',
 '불친절',
 '테이블',
 '간격',
 '정도',
 '편안',
 '각종',
 '굿즈',
 '북카페',
 '혼자',
 '한잔',
 '아이쇼핑',
 '일석이조',
 '커피',
 '가격',
 '대가',
 '테이블',
 '간격',
 '북카페',
 '편집',
 '느낌',
 '진열',
 '물품',
 '커피',
 '공간',
 '디자인',
 '요소',
 '매장',
 '차지',
 '공부',
 '작업',
 '커피',
 '때문',
 '입도',
 '음료',
 '테이크아웃',
 '하프',
 '음료',
 '라떼',
 '밍밍',
 '팝업',
 '독립',
 '적도',
 '진열',
 '헤드셋',
 '준비',
 '판매',
 '제품',
 '학생',
 '프로젝트',
 '회의',
 '자주',
 '방

In [28]:
## 가중치 계산
# Create a dictionary of synonym groups
synonym_groups = {
    "western": ["양식","피자","파스타","버거","샐러드","스테이크","화덕","시카고피자","레스토랑","피클"],
    "japan": ["일식","라멘","우동","튀김","초밥","츠케멘","소바","차슈","카츠","회"],
    "service": ["서비스","친절","퀄리티","위생","설명","상태","관리","청결","정성","합리"],
    "location": ["스타일","컨셉","인스타","특색","감성","뷰","테라스","테마","야외","예술"],
    "meat": ["고기","곱창","돼지","닭갈비","등심","갈비","대창","삼겹살","육즙","양고기"],
    "alchol": ["술","국물","안주","맥주","술집","막걸리","소주","한잔","생맥주", "칵테일"],
    "cafe": ["카페","딸기","케이크","음료","빵","디저트","라떼","빙수","케익","생크림"],
    "time": ["시간","근처","거리","동네","예약","대기","웨이팅","주변","멀리","가까"]
}

# Function to calculate the frequency of group words in each review's word list
def calculate_group_weights(words, group_words):
    return sum(word in group_words for word in words)

# Apply the function for each group
for group_name, group_words in synonym_groups.items():
    review_df_clean[group_name] = review_df_clean['filtered_words'].apply(lambda words: calculate_group_weights(words, group_words))

# Aggregate these group weights by merchant
grouped_weights = review_df_clean.groupby('merchant')[list(synonym_groups.keys())].sum()

# Normalize the weights by the number of reviews for each restaurant
normalized_weights = grouped_weights.div(review_df_clean.groupby('merchant').size(), axis=0)

# Display the processed weights for each restaurant
normalized_weights.reset_index().head()

C:\Users\Jws\AppData\Local\Temp\ipykernel_32960\4274761084.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df_clean[group_name] = review_df_clean['filtered_words'].apply(lambda words: calculate_group_weights(words, group_words))
C:\Users\Jws\AppData\Local\Temp\ipykernel_32960\4274761084.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df_clean[group_name] = review_df_clean['filtered_words'].apply(lambda words: calculate_group_weights(words, group_words))
C:\Users\Jws\AppData\Local\Tem

,merchant,western,japan,service,location,meat,alchol,cafe,time
0,1984,0.000000,0.000000,0.063492,0.222222,0.000000,0.015873,0.571429,0.158730
1,21세기 우리바다수산,0.000000,0.090909,0.181818,0.000000,0.000000,0.022727,0.000000,0.272727
2,38도씨식당,0.307692,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000
3,943킹스크로스,0.000000,0.006211,0.031056,0.111801,0.003106,0.074534,0.549689,0.055901
4,가로수포차,0.333333,0.000000,0.250000,0.083333,0.083333,0.416667,0.000000,0.000000


In [31]:
# Reset index to ensure 'merchant' is a column
normalized_weights_reset = normalized_weights.reset_index()

In [57]:
normalized_weights_reset['별천지설농탕']

KeyError: '별천지설농탕'

In [33]:
# normalized_weights_reset.to_csv('../data2/weights.csv', index=False)

In [34]:
from scipy.stats import pearsonr
import numpy as np

In [35]:
### 유사도 비교


# Function to calculate Pearson correlation matrix for all pairs of merchants
def pearson_correlation_matrix(df):
    # Number of merchants
    n = df.shape[0]
    # Initialize the correlation matrix
    correlation_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                data_i = df.iloc[i, 1:].values
                data_j = df.iloc[j, 1:].values
                correlation, _ = pearsonr(data_i, data_j)
                correlation_matrix[i, j] = correlation
            else:
                correlation_matrix[i, j] = 1  # Self correlation is always 1
    return correlation_matrix

# Calculate the Pearson correlation matrix
correlation_matrix = pearson_correlation_matrix(normalized_weights_reset)
correlation_matrix

array([[ 1.        , -0.1258035 , -0.29907038, ..., -0.31724098,
        -0.3732846 , -0.20629136],
       [-0.1258035 ,  1.        , -0.16890769, ..., -0.16045149,
         0.23341702,  0.50467209],
       [-0.29907038, -0.16890769,  1.        , ..., -0.2372895 ,
        -0.24557394, -0.07057202],
       ...,
       [-0.31724098, -0.16045149, -0.2372895 , ...,  1.        ,
         0.03738847, -0.29740896],
       [-0.3732846 ,  0.23341702, -0.24557394, ...,  0.03738847,
         1.        , -0.04373735],
       [-0.20629136,  0.50467209, -0.07057202, ..., -0.29740896,
        -0.04373735,  1.        ]])

In [44]:
import numpy as np

In [45]:
np.save('../data2/cor.npy', correlation_matrix)

In [46]:
## 유사도 높은 10개 식당 출력
def find_similar_restaurants(restaurant_name, weights_path, cor_path, top_n=10):
    # 데이터 로드
    weights_df = pd.read_csv(weights_path)
    cor_matrix = np.load(cor_path)
    
    # 음식점 이름 목록 가져오기
    restaurant_names = weights_df['merchant'].values
    
    # 주어진 음식점의 인덱스 찾기
    if restaurant_name in restaurant_names:
        idx = np.where(restaurant_names == restaurant_name)[0][0]
        
        # 유사도 점수와 인덱스 가져오기
        similarity_scores = cor_matrix[idx]
        
        # 유사도에 따라 인덱스 정렬 및 상위 N개 선택
        sorted_indices = np.argsort(similarity_scores)[::-1]
        top_indices = sorted_indices[1:top_n+1]  # 자기 자신 제외
        
        # 결과 출력
        print(f"Top {top_n} restaurants similar to {restaurant_name}:")
        for i in top_indices:
            print(f"{restaurant_names[i]} - Similarity: {similarity_scores[i]}")
    else:
        print(f"Restaurant name '{restaurant_name}' not found in the list.")

# 사용 예
# find_similar_restaurants('example_restaurant', 'path/to/weights.csv', 'path/to/cor.npy')

In [56]:
find_similar_restaurants('삼거리포차','../data2/weights.csv','../data2/cor.npy')

Top 10 restaurants similar to 삼거리포차:
꼼보포차 홍대2호점 - Similarity: 0.9861264954268181
룸넘버3 브런치 카페&와인 - Similarity: 0.9773555548504418
미로식당 - Similarity: 0.9759046823277522
산울림1992 - Similarity: 0.9656821497458467
키즈나 - Similarity: 0.9519366412056718
벽돌포차 - Similarity: 0.9439794899424117
코키펍 홍대점 - Similarity: 0.9169493006161777
썰스데이파티 - Similarity: 0.9169493006161777
어른이대공원 - Similarity: 0.8988714651559209
별천지설농탕 - Similarity: 0.8960632943213851


In [ ]:
## -------------------------------------------------------

In [2]:
!pip install scipy

In [3]:
!pip install cython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
    --------------------------------------- 0.0/2.8 MB 495.5 kB/s eta 0:00:06
   ---- ----------------------------------- 0.3/2.8 MB 2.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.8 MB 2.2 MB/s eta 0:00:02
   -------- ------------------------------- 0.6/2.8 MB 2.5 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.8 MB 3.7 MB/s eta 0:00:01
   --------------------------------- ------ 2.4/2.8 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 8.5 MB/s eta 0:00:00


In [15]:
import numpy as np
import cython
import scipy

In [ ]:
# !pip install gensim

In [16]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText

In [ ]:
from gensim.models import KeyedVectors

# 모델 로드
model = KeyedVectors.load_word2vec_format('./cc.ko.300.bin', binary=True)

In [3]:
from gensim.models import KeyedVectors
from gensim import models

# 모델 로드
ko_model = models.fasttext.load_facebook_model('./cc.ko.300.bin')

# 'filtered_words'에서 상위 단어 선택
top_words = ['라멘', '양식', '서비스', '커플', '고기', '술', '카페', '웨이팅', '데이트', '가족']  # 예시 단어 리스트

# 각 단어에 대해 유사 단어 찾기
for word in top_words:
    if word in ko_model.wv.key_to_index:  # 모델에 단어가 있는지 확인
        similar_words = ko_model.wv.most_similar(word, topn=13)  # 가장 유사한 10개 단어 찾기
        print(f"Word: {word}, Similar Words: {similar_words}")
    else:
        print(f"{word} is not in the vocabulary")

Word: 라멘, Similar Words: [('소바', 0.6055879592895508), ('라멘의', 0.6014007925987244), ('퓨얼드', 0.5990808010101318), ('라멘은', 0.5769829750061035), ('이치란', 0.576982319355011), ('라멘으로', 0.5695579051971436), ('초밥', 0.5679571628570557), ('잇푸도', 0.5663458108901978), ('덮밥', 0.5654603242874146), ('오뎅', 0.5640437602996826)]
Word: 양식, Similar Words: [('糧食', 0.7907487750053406), ('良識', 0.7743086814880371), ('洋食', 0.6927233934402466), ('樣式', 0.6794001460075378), ('양식과', 0.5734102725982666), ('수산증', 0.5720512866973877), ('일용할', 0.5660973787307739), ('양식의', 0.552295982837677), ('양식은', 0.5503145456314087), ('보자르', 0.5492727756500244)]
Word: 서비스, Similar Words: [('서비스의', 0.6431233286857605), ('호텔식당조직의', 0.6139702200889587), ('서비스를', 0.6111576557159424), ('이용과정', 0.5958628058433533), ('서비스와', 0.5880147218704224), ('-서비스', 0.5860386490821838), ('광고서비스', 0.58290696144104), ('U-도서관', 0.5818356275558472), ('콘시어지', 0.5796222686767578), ('제공업체의', 0.5785194635391235)]
Word: 커플, Similar Words: [('커플과', 0.5707965493